In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive

/content/drive/MyDrive


In [ ]:
!pip install SimpleITK

     |████████████████████████████████| 48.4 MB 20 kB/s 


In [ ]:
import os
import json
import pandas as pd
import numpy as np

#CLINICAL_NAMES = ['SubjectId', 'age', 'bmi', 'gender', 'gender_num', 'comorbidities', 'smoking_history', 'radiographic_size', 'preop_egfr',
                  #'pathology_t_stage', 'pathology_n_stage', 'pathology_m_stage','age_when_quit_smoking','pack_years', 'grade', 'aua_risk_group', 'task_1_label', 'task_2_label']

#"age_at_nephrectomy"
#"gender"
#"body_mass_index"
#"comorbidities"
#"smoking_history"
#"age_when_quit_smoking"
#"pack_years"
#"chewing_tobacco_use"
#"alcohol_use"
#"last_preop_egfr"
#"radiographic_size"
#"voxel_spacing"

CLINICAL_NAMES = ['SubjectId', 
                  'age', 
                  'bmi', 
                  'gender', 
                  'gender_num', 
                  'comorbidities', 
                  'smoking_history', 
                  'radiographic_size', 
                  'preop_egfr',
                  #'pathology_t_stage', 
                  #'pathology_n_stage', 
                  #'pathology_m_stage',
                  'age_when_quit_smoking',
                  'alcohol_use',
                  #'chewing_tobacco_use',
                  'pack_years', 
                  #'grade', 
                  'aua_risk_group', 'task_1_label', 'task_2_label']

# "age_at_nephrectomy"
# "gender"
# "body_mass_index"
# "comorbidities"
# "smoking_history"
# "age_when_quit_smoking"
# "pack_years"
# "chewing_tobacco_use"
# "alcohol_use"
# "last_preop_egfr"
# "radiographic_size"
# "voxel_spacing"


def create_knight_clinical(original_file, processed_file=None):
    with open(original_file) as f:
        clinical_data = json.load(f)
    t_stage_count = np.zeros((5))
    aua_risk_count = np.zeros((5))
    df = pd.DataFrame(columns=CLINICAL_NAMES)
    for index, patient in enumerate(clinical_data):
        df.loc[index, 'SubjectId'] = patient['case_id']
        df.loc[index, 'age'] = patient['age_at_nephrectomy']
        df.loc[index, 'bmi'] = patient['body_mass_index']

        df.loc[index, 'gender'] = patient['gender']
        if patient['gender'] == 'male':    # 0:'male'  1:'female','transgender_male_to_female'
            df.loc[index, 'gender_num'] = 0
        else:
            df.loc[index, 'gender_num'] = 1

        df.loc[index, 'comorbidities'] = 0    # 0:no_comorbidities 1:comorbidities_exist
        for key, value in patient['comorbidities'].items():
            if value:
                df.loc[index, 'comorbidities'] = 1

        df.loc[index, 'smoking_history'] = patient['smoking_history']
        if patient['smoking_history']=='never_smoked':    # 0:'never_smoked' 1:'previous_smoker'  2:'current_smoker'
            df.loc[index, 'smoking_history'] = 0
        elif patient['smoking_history'] == 'previous_smoker':
            df.loc[index, 'smoking_history'] = 1
        elif patient['smoking_history'] == 'current_smoker':
            df.loc[index, 'smoking_history'] = 2

        df.loc[index, 'radiographic_size'] = patient['radiographic_size']
        if patient['last_preop_egfr']['value'] == '>=90':
            df.loc[index, 'preop_egfr'] = 90
        else:
            df.loc[index, 'preop_egfr'] = patient['last_preop_egfr']['value']

        #df.loc[index, 'pathology_t_stage'] = patient['pathology_t_stage']
        #df.loc[index, 'pathology_n_stage'] = patient['pathology_n_stage']
        #df.loc[index, 'pathology_m_stage'] = patient['pathology_m_stage']
        df.loc[index, 'aua_risk_group'] = patient['aua_risk_group']
       
        # Task 1 labels:
        if patient['aua_risk_group'] in ['high_risk', 'very_high_risk']:    # 1:'3','4'  0:'0','1a','1b','2a','2b'
            df.loc[index, 'task_1_label'] = 1 # CanAT
        else:
            df.loc[index, 'task_1_label'] = 0 # NoAT

        # Task 2 labels:
        if patient['aua_risk_group']=='benign':
            df.loc[index, 'task_2_label'] = 0 
        elif patient['aua_risk_group']=='low_risk':
            df.loc[index, 'task_2_label'] = 1
        elif  patient['aua_risk_group']=='intermediate_risk':
            df.loc[index, 'task_2_label'] = 2
        elif patient['aua_risk_group']=='high_risk':
            df.loc[index, 'task_2_label'] = 3
        elif patient['aua_risk_group']=='very_high_risk':
            df.loc[index, 'task_2_label'] = 4
        else:
            ValueError('Wrong risk class')

        # former classification - deprecated
        #if patient['pathology_t_stage'] in ['3', '4']:    # 1:'3','4'  0:'0','1a','1b','2a','2b'
        #    df.loc[index, 'pathology_t_stage_classify'] = 1
        #else:
        #    df.loc[index, 'pathology_t_stage_classify'] = 0
        #t_stage = int(patient['pathology_t_stage'][0])
        #t_stage_count[t_stage] += 1
        aua_risk = int(df.loc[index, 'task_2_label'])
        aua_risk_count[aua_risk] += 1
        #df.loc[index, 'grade'] = patient['tumor_isup_grade']
        ##### new parameters
        #df.loc[index,'alcohol_use']=patient['alcohol_use']
        if patient['alcohol_use']=='two_or_less_daily':
            df.loc[index, 'alcohol_use'] = 1
        elif patient['alcohol_use']=='never_or_not_in_last_3mo':
            df.loc[index, 'alcohol_use'] = 2
        elif patient['alcohol_use']=='more_than_two_daily':
            df.loc[index, 'alcohol_use'] = 3
         
        df.loc[index,'pack_years']=patient['pack_years']
        #df.loc[index,'age_when_quit_smoking']=patient['age_when_quit_smoking']
        if patient['age_when_quit_smoking']=='not_applicable':
            df.loc[index, 'age_when_quit_smoking'] = 1

        #df.loc[index,'achewing_tobacco_use']=patient['chewing_tobacco_use']
        #age_when_quit_smoking

    if processed_file is not None:
        # save csv file
        df.to_csv(processed_file, index=False)
        df = df.drop(['gender', 'pathology_t_stage', 'pathology_n_stage', 'pathology_m_stage'], axis=1)
        df.to_csv(os.path.splitext(processed_file)[0] + '_numeric.csv' , index=False)
    print(f'Pathology t-stage count summary: {t_stage_count}')
    print(f'AUA risk count summary: {aua_risk_count}')
    return df


In [ ]:
#original_file='C:\\Users\\Administrateur\\Desktop\\testfused_model\\knight.json'
original_file='/content/drive/MyDrive/knight_challenegs2022/knight.json'
df=create_knight_clinical(original_file, processed_file=None)
#df.to_csv('processed_file.csv', index=False)

Pathology t-stage count summary: [0. 0. 0. 0. 0.]
AUA risk count summary: [ 25. 142.  48.  40.  45.]


In [ ]:
#preop_egfr
#df['preop_egfr'].unique()
df['preop_egfr']=df['preop_egfr'].fillna(77)
df['alcohol_use']=df['alcohol_use'].fillna(0)
df['pack_years']=df['pack_years'].fillna(0)
df['age_when_quit_smoking']=df['age_when_quit_smoking'].fillna(0)
#df["bmi"] = df['bmi'] /50.0
#df['age']=df['age']/120.0
##df['preop_egfr']=df['preop_egfr']/90.0
#df['radiographic_size']=df['radiographic_size']/15.0
#df['alcohol_use']=df['alcohol_use']/df['alcohol_use'].max()
#df['pack_years']=df['pack_years'].astype(float) / df['pack_years'].max()

In [ ]:
df.drop(['gender'],axis=1, inplace=True)

In [ ]:
df.drop(['aua_risk_group'],axis=1, inplace=True)

In [ ]:
df

,SubjectId,age,bmi,gender_num,comorbidities,smoking_history,radiographic_size,preop_egfr,age_when_quit_smoking,alcohol_use,pack_years,task_1_label,task_2_label
0,case_00000,49,29.47,0,0,0,2.4,77.0,1,1,0,0,2
1,case_00001,50,33.71,0,0,1,2.2,67.0,0,1,7,0,1
2,case_00002,74,28.13,0,1,2,4.5,77.0,1,2,50,1,3
3,case_00003,44,38.69,1,0,1,3.0,86.0,0,1,0,0,1
4,case_00004,73,28.59,0,1,1,3.0,77.0,0,2,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,case_00295,48,43.9,0,0,2,3.3,87.0,1,1,0,0,1
296,case_00296,51,17.8,1,0,1,2.8,90.0,0,2,0,0,1
297,case_00297,64,23.37,0,1,1,4.2,50.0,0,2,0,0,2
298,case_00298,40,22.45,0,0,1,5.9,77.0,0,1,0,1,3


In [ ]:
import pandas as pd
#path='C:\\Users\\Administrateur\\Desktop\\testfused_model\\fuse-med-ml-master\\fuse-med-ml-master\\baseline'
splits=pd.read_pickle('/content/drive/MyDrive/knight_challenegs2022/splits_final.pkl')
# For this example, we use split 0 out of the 5 available cross validation splits
split = splits[4]
train=split['train']
pdftrain=pd.DataFrame(train).rename(columns={0:'SubjectId'})


val=split['val']
pdfval=pd.DataFrame(val).rename(columns={0:'SubjectId'})


#data_dir='C:\\Users\\Administrateur\\Desktop\\testfused_model'
#train_data_source = FuseDataSourceDefault(list(split['train']))
# name=pdftrain['SubjectId']
# imageurlc1=pathfile.loc[pathfile['SubjectId']==name]
# data1=[]
# for i in name:
#     print(i)
#     data=pathfile.loc[pathfile['SubjectId']==i]
#     print(data)
#     data1.append(data)
# import numpy as np
# data2=np.array(data1)
# data3=np.squeeze(data2,axis=1)
# traindata=pd.DataFrame(data3)
pathfile=df
traindatapd = pd.merge(pathfile, pdftrain, on=['SubjectId'], how='inner')
traindatapd .head()

valdatapd = pd.merge(pathfile, pdfval, on=['SubjectId'], how='inner')
valdatapd.head()

#valdatapd.to_csv('validfold0_unnormalized.csv',index=False)
#traindatapd.to_csv('trainfold0_unnormalized.csv',index=False)

,SubjectId,age,bmi,gender_num,comorbidities,smoking_history,radiographic_size,preop_egfr,age_when_quit_smoking,alcohol_use,pack_years,task_1_label,task_2_label
0,case_00005,35,35.96,0,1,2,4.9,90.0,1,2,0,1,3
1,case_00012,62,30.92,1,1,0,9.59,42.0,1,1,0,1,4
2,case_00022,66,30.69,1,0,0,2.3,80.0,1,3,0,0,1
3,case_00023,49,34.53,0,0,0,2.3,90.0,1,2,0,0,1
4,case_00029,68,23.4,1,1,2,5.4,82.0,1,1,0,0,2


In [ ]:
valdatapd.to_csv('validfold4_unnormalized.csv',index=False)
traindatapd.to_csv('trainfold4_unnormalized.csv',index=False)

In [ ]:
ls

clinical_dataknight.csv  models/           trainfold0.csv
knight.json              splits_final.pkl  validfold0.csv


In [ ]:
valdatapd.to_csv('validfold0_unnormalized.csv',index=False)
traindatapd.to_csv('trainfold0_unnormalized.csv',index=False)

In [ ]:
cd /content/drive/MyDrive/knight_challenegs2022

/content/drive/MyDrive/knight_challenegs2022


In [ ]:
!mkdir models

In [ ]:
cd /content/drive/MyDrive/knight_challenegs2022/models

/content/drive/MyDrive/knight_challenegs2022/models


In [ ]:
cd /content/drive/MyDrive/knight_challenegs2022/models

/content/drive/MyDrive/knight_challenegs2022/models


In [ ]:
!mkdir mlpmodel

In [ ]:
cd /content/drive/MyDrive/knight_challenegs2022

/content/drive/MyDrive/knight_challenegs2022


In [ ]:
import torch
import pandas as pd
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import os
import SimpleITK as sitk
import numpy as np
import os
import numpy as np
import skimage
#import skimage.io as io
import skimage.transform as transform
import torch

import numpy as np
import pandas as pd
import torch
import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "0,1" 
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import os
import cv2
import matplotlib.pyplot as plt
import skimage 

#import logging
#import traceback
from typing import Optional, Tuple


def kits_normalization(input_image: np.ndarray):
    # first, clip to [-62, 310] (corresponds to 0.5 and 99.5 percentile in the foreground regions)
    # then, subtract 104.9 and divide by 75.3 (corresponds to mean and std in the foreground regions, respectively)
    clip_min = -62
    clip_max = 301
    mean_val = 104.0
    std_val = 75.3
    input_image = np.minimum(np.maximum(input_image, clip_min), clip_max)
    input_image -= mean_val
    input_image /= std_val
    return input_image

def normalize_to_range(input_image: np.ndarray, range: Tuple = (0.0, 1.0)):
    """
    Scales tensor to range
    @param input_image: image of shape (H x W x C)
    @param range:       bounds for normalization
    @return:            normalized image
    """
    max_val = input_image.max()
    min_val = input_image.min()
    if min_val == max_val == 0:
        return input_image
    input_image = input_image - min_val
    input_image = input_image / (max_val - min_val)
    input_image = input_image * (range[1] - range[0])
    input_image = input_image + range[0]
    return input_image

from typing import Iterable

def center_crop(np_image: np.ndarray,
                new_shape: Iterable[int],
                outside_val: float = 0
                ) -> np.ndarray:
    output_image = np.full(new_shape, outside_val, np_image.dtype)

    slices = tuple()
    offsets = tuple()
    for it, sh in enumerate(new_shape):
        size = sh // 2
        if it == 0:
            center = np_image.shape[it] - size
        else:
            center = (np_image.shape[it] // 2)
        start = center - size
        stop = center + size + (sh % 2)

        # computing what area of the original image will be in the cropped output
        slce = slice(max(0, start), min(np_image.shape[it], stop))
        slices += (slce,)

        # computing offset to pad if the crop is partly outside of the scan
        offset = slice(-min(0, start), 2 * size - max(0, (start + 2 * size) - np_image.shape[it]))
        offsets += (offset,)

    output_image[offsets] = np_image[slices]

    return output_image


def pad_image(image: np.ndarray, outer_height: int, outer_width: int, pad_value: Tuple):
    """
    Pastes input image in the middle of a larger one
    @param image:        image of shape (H x W x C)
    @param outer_height: final outer height
    @param outer_width:  final outer width
    @param pad_value:    value for padding around inner image
    @return:             padded image
    """
    inner_height, inner_width = image.shape[0], image.shape[1]
    h_offset = int((outer_height - inner_height) / 2.0)
    w_offset = int((outer_width - inner_width) / 2.0)
    outer_image = np.ones((outer_height, outer_width, 3), dtype=image.dtype) * pad_value
    outer_image[h_offset:h_offset + inner_height, w_offset:w_offset + inner_width, :] = image

    return outer_image


class deeeeset(Dataset):
    def __init__(self,in_out):
        self.in_out=in_out
        
        #self.new_shape=new_shape
        #self.input_data = input_data
        self.ids=self.in_out['SubjectId']
        self.label=self.in_out['task_1_label']
        
        self.feature=self.in_out.drop(['SubjectId','task_1_label','task_2_label'],axis=1, inplace=True)

    
            
        
    def __getitem__(self,idx):
        
        feat=self.in_out.iloc[idx]
        x_feature=pd.DataFrame(feat).T
        
        # norm_img=get_array_img
        
        y_output=self.label[idx]
        x_f_array=np.array(x_feature)
        x_f_array_t=torch.from_numpy(x_f_array).float()
        x_f_array_t=torch.squeeze(x_f_array_t,axis=0)
        return (x_f_array_t,y_output)
    
    def __len__(self):
        return(len(self.label))
    


#input_image = center_crop(input_image, new_shape=new_shape) 
    
    
filet=pd.read_csv('/content/drive/MyDrive/knight_challenegs2022/trainfold0_unnormalized.csv')
#data='/home/imranr/knights/data'

filev=pd.read_csv('/content/drive/MyDrive/knight_challenegs2022/validfold0_unnormalized.csv')

len(filev['task_1_label']==0)
len(filev['task_1_label']==1)

#classes=['NoAT','CanAT']
c1=filet['task_1_label'].value_counts()[0]
#175

#c2=filet['task_1_label'].value_counts()[1]
#65

train_dataset=deeeeset(filet)
valid_dataset=deeeeset(filev)

print(len(train_dataset))
print(len(valid_dataset))

f,o=valid_dataset[2]

#print(i.shape)
#print(i.min())
#print(i.max())
print(o)
print(f)

# i,f, o=train_dataset[2]

# print(i.shape)
# print(i.min())
# print(i.max())
# print(o)
# print(f)

#%
from torch.utils.data import DataLoader

# train_dataloader = DataLoader(ob_d, batch_size=4, shuffle=True)
# #test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)
# for i, data in enumerate(train_dataloader):
#     img,f,l=data
#     print(img.shape)
#     print(f.shape)
#     print(l)
#     break
    

batch=4

train_dataloader = DataLoader(train_dataset, batch_size= batch,pin_memory=True,num_workers=6, shuffle=True)

valid_dataloader = DataLoader(valid_dataset, batch_size= batch,pin_memory=True,num_workers=6, shuffle=False)


import sys 
import os
import glob
import time
import random
import os
import glob
import time
import random
import torch
import numpy as np
import pandas as pd
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Copyright (c) MONAI Consortium
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#     http://www.apache.org/licenses/LICENSE-2.0
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

"""
(C) Copyright 2021 IBM Corp.
Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at
   http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
Created on June 30, 2021
"""

from typing import Tuple, Any

import torch.nn as nn
from torch import Tensor
from torch.hub import load_state_dict_from_url
from torchvision.models.video.resnet import VideoResNet, BasicBlock, Conv3DSimple, BasicStem, model_urls


class FuseBackboneResnet3D(VideoResNet):
    """
    3D model classifier (ResNet architecture"
    """

    def __init__(self, pretrained: bool = False, in_channels: int = 1, name: str = "r3d_18") -> None:
        """
        Create 3D ResNet model
        :param pretrained: Use pretrained weights
        :param in_channels: number of input channels
        :param name: model name. currently only 'r3d_18' is supported
        """
        # init parameters per required backbone
        init_parameters = {
            'r3d_18': {'block': BasicBlock,
                       'conv_makers': [Conv3DSimple] * 4,
                       'layers': [2, 2, 2, 2],
                       'stem': BasicStem},
        }[name]
        # init original model
        super().__init__(**init_parameters)

        # load pretrained parameters if required
        if pretrained:
            state_dict = load_state_dict_from_url(model_urls[name])
            self.load_state_dict(state_dict)

        # save input parameters
        self.pretrained = pretrained
        self.in_channels = in_channels
        # override the first convolution layer to support any number of input channels
        self.stem = nn.Sequential(
            nn.Conv3d(self.in_channels, 64, kernel_size=(3, 7, 7), stride=(1, 2, 2),
                      padding=(1, 3, 3), bias=False),
            nn.BatchNorm3d(64),
            nn.ReLU(inplace=True)
        )

    def features(self, x: Tensor) -> Any:
        """
        Extract spatial features - given a 3D tensor
        :param x: Input tensor - shape: [batch_size, channels, z, y, x]
        :return: spatial features - shape [batch_size, n_features, z', y', x']
        """
        x = self.stem(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        return x

    def forward(self, x: Tensor) -> Tuple[Tensor, None, None, None]:  # type: ignore
        """
        Forward pass. 3D global classification given a volume
        :param x: Input volume. shape: [batch_size, channels, z, y, x]
        :return: logits for global classification. shape: [batch_size, n_classes].
        """
        x = self.features(x)
        return x
    
backbone=FuseBackboneResnet3D(pretrained=True)   
from typing import Optional, Sequence
import torch.nn as nn

class ClassifierMLP(nn.Module):
    def __init__(self, in_ch: int, num_classes: Optional[int], layers_description: Sequence[int]=(256,128), dropout_rate: float = 0.1):
        super().__init__()
        layer_list = []
        layer_list.append(nn.Linear(in_ch, layers_description[0]))
        layer_list.append(nn.ReLU())
        if dropout_rate is not None and dropout_rate > 0:
            layer_list.append(nn.Dropout(p=dropout_rate))
        last_layer_size = layers_description[0]
        for curr_layer_size in layers_description[1:]:
            layer_list.append(nn.Linear(last_layer_size, curr_layer_size))
            layer_list.append(nn.ReLU())
            if dropout_rate is not None and dropout_rate > 0:
                layer_list.append(nn.Dropout(p=dropout_rate))
            last_layer_size = curr_layer_size
        
        if num_classes is not None:
            layer_list.append(nn.Linear(last_layer_size, num_classes))
        
        self.classifier = nn.Sequential(*layer_list)

    def forward(self, x):
        x = self.classifier(x)
        return x
  
model=ClassifierMLP(10,2)  
import torch    

inp=torch.rand(1,10)
out=model(inp)
print(out.shape)


model=nn.DataParallel(model)
model=model.to(device)
#loss_func=nn.BCEWithLogitsLoss()
from tqdm import tqdm
import torch.optim as optim
optimizer=optim.Adam(model.parameters(),lr=0.00001)

#classes=['NRG','RG']
classes=['NoAT','CanAT']
#c1=data['label'].value_counts()[0]
#c2=data['label'].value_counts()[1]
my_distribution=np.array([175,65])
class_weights = torch.from_numpy(np.divide(1, my_distribution)).float().to(device)
class_weights = class_weights / class_weights.sum()
for i, c in enumerate(classes):
  print('Weight for class %s: %f' % (c, class_weights.cpu().numpy()[i]))
loss_func = nn.CrossEntropyLoss(weight=class_weights)
#label=label.to(torch.int64)
################################ training functions ###################
def train_fn(model,train_loader):
    model.train()
    counter=0
    training_run_loss=0.0
    train_running_correct=0.0
    for i, data in tqdm(enumerate(train_loader),total=int(len(train_dataset)/train_loader.batch_size)):
        counter+=1
        # extract dataset
        feature,label=data
        #imge=imge.float()
        #label=label.float()
        label.to(torch.int64)
        feature=feature.float()
        feature=feature.to(device)
        #imge=imge.to(device)
        label=label.to(device)
        #imge=imge.cuda()
        #label=label.cuda()
        # zero_out the gradient
        optimizer.zero_grad()
        output=model(feature)
        loss=loss_func(output,label)
        training_run_loss+=loss.item()
        _,preds=torch.max(output.data,1)
        train_running_correct+=(preds==label).sum().item()
        loss.backward()
        optimizer.step()
    ###################### state computation ###################
    train_loss=training_run_loss/len(train_loader.dataset)
    train_loss_ep.append(train_loss)
    train_accuracy=100.* train_running_correct/len(train_loader.dataset)
    train_accuracy_ep.append(train_accuracy)
    print(f"Train Loss:{train_loss:.4f}, Train Acc:{train_accuracy:0.2f}")
    return train_loss_ep,train_accuracy_ep

########################## validation function ##################
def validation_fn(model,valid_loader):
  # evluation start
    print("validation start")
    
    model.eval()
    val_running_loss = 0.0
    val_running_correct = 0
    with torch.no_grad():
        for i,data in tqdm(enumerate(valid_loader),total=int(len(valid_loader)/valid_loader.batch_size)):
            feature,label=data
            #imge=imge.float()
            #label=label.float()
            label.to(torch.int64)
            feature=feature.float()
            feature=feature.to(device)
            #imge=imge.to(device)
            label=label.to(device)
            #imge=imge.cuda()
            #label=label.cuda()
            output=model(feature)
            loss=loss_func(output,label)
            val_running_loss+=loss.item()
            _,pred=torch.max(output.data,1)
            val_running_correct+=(pred==label).sum().item()
        val_loss=val_running_loss/len(valid_loader.dataset)
        val_loss_ep.append(val_loss)
        val_accuracy=100.* val_running_correct/(len(valid_loader.dataset))
        val_accuracy_ep.append(val_accuracy)
        print(f"Val Loss:{val_loss:0.4f}, Val_Acc:{val_accuracy:0.2f}")
        return val_loss_ep,val_accuracy_ep

import torch.optim as optim
optimizer=optim.Adam(model.parameters(),lr=0.0001)
train_loss_ep=[]
train_accuracy_ep=[]
val_loss_ep=[]
val_accuracy_ep=[]
lr = 3e-4
log = pd.DataFrame(index=[], columns=['epoch', 'lr', 'loss', 'accu', 'val_loss', 'val_accu'])
early_stop=20
epochs=500
best_acc = 0
name='mlpmodel'
trigger = 0
for epoch in range(epochs):
    print('Epoch [%d/%d]' %(epoch, epochs))
    # train for one epoch
    train_loss_ep,train_accuracy_ep=train_fn(model,train_dataloader)
    train_loss_ep1=np.mean(train_loss_ep)
    train_accuracy_ep1=np.mean(train_accuracy_ep)
    #y_pred,labels=Prediciton_fn(model,valid_loader)

    val_loss_ep,val_accuracy_ep=validation_fn(model,valid_dataloader)
    val_loss_ep1=np.mean(val_loss_ep)
    val_accuracy_ep1=np.mean(val_accuracy_ep)
    
    print('loss %.4f - accu %.4f - val_loss %.4f - val_accu %.4f'%(train_loss_ep1, train_accuracy_ep1, val_loss_ep1, val_accuracy_ep1))

    tmp = pd.Series([epoch,lr,train_loss_ep1,train_accuracy_ep1,val_loss_ep1,val_accuracy_ep1], index=['epoch', 'lr', 'loss', 'accu', 'val_loss', 'val_accu'])

    log = log.append(tmp, ignore_index=True)
    log.to_csv('models/%s/log.csv' %name, index=False)

    trigger += 1

    if val_accuracy_ep1 > best_acc:
        torch.save(model.state_dict(), 'models/%s/3dmodeltask1.pth' %name)
        best_acc = val_accuracy_ep1
        print("=> saved best model")
        trigger = 0

    # early stopping
    if not early_stop is None:
        if trigger >= early_stop:
            print("=> early stopping")
            break

    torch.cuda.empty_cache() 

240
60
0
tensor([53.0000, 31.6100,  1.0000,  0.0000,  0.0000,  5.0000, 77.0000,  1.0000,
         1.0000,  0.0000])


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


torch.Size([1, 2])
Weight for class NoAT: 0.270833
Weight for class CanAT: 0.729167
Epoch [0/500]


100%|██████████| 60/60 [00:00<00:00, 96.81it/s] 

Train Loss:0.3066, Train Acc:55.83
validation start


...

Val Loss:0.1112, Val_Acc:81.67
loss 0.1324 - accu 75.6950 - val_loss 0.1158 - val_accu 81.3866
Epoch [244/500]



100%|██████████| 60/60 [00:00<00:00, 90.10it/s] 

Train Loss:0.1080, Train Acc:79.58
validation start



15it [00:00, 60.19it/s]

Val Loss:0.1105, Val_Acc:81.67
loss 0.1323 - accu 75.7109 - val_loss 0.1158 - val_accu 81.3878
=> early stopping
